In [1]:
import dateutil
import datetime as dt
import time
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from scipy import signal
import datetime as dt
import pylab
from sklearn.linear_model import LinearRegression
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
warnings.filterwarnings("ignore")
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
df_set = pd.read_excel('TR_Sets.xlsx')

In [3]:
df = pd.read_csv('filtered_joined_df_.csv')

In [4]:
customers_names = pd.read_csv('sellin_customers_1.csv')
customers_names = customers_names.set_index('CUSTOMER_ID')
customers_names = customers_names[['NCUS']]
df = df.join(customers_names, on='CUSTOMER_ID')
df = df.loc[df['Q_BIL_GROSAL'] > 0]

In [6]:
df2 = df.groupby(['LFORSUBARE','CITE']).aggregate({'Q_BIL_GROSAL':'count'})

In [7]:
cites_sup3 = set(df2.loc[df2['Q_BIL_GROSAL'] > 3].index.get_level_values(1))

In [9]:
df = df.loc[df['CITE'].isin(cites_sup3)]

In [11]:
df_grouped_cite = df.groupby(['LFORSUBARE', 'CITE','Year','Month']).aggregate({'Year':'mean','Month':'mean','Q_BIL_GROSAL':'sum'})
df_grouped_cite['Date'] = pd.to_datetime(df_grouped_cite['Year'].astype(str) + '-'+df_grouped_cite['Month'].astype(str) +'-01')
df_grouped_cite = df_grouped_cite.drop(columns =['Year','Month'])

Q_BIL_GROSAL       Date
CITE    Year Month                         
G002859 2019 11             2150 2019-11-01
             12             2350 2019-12-01
G010034 2016 1                 5 2016-01-01
             2                 6 2016-02-01
             6                 1 2016-06-01
...                          ...        ...
G064203 2018 2                 2 2018-02-01
             3                 3 2018-03-01
             7                 4 2018-07-01
        2019 7                 4 2019-07-01
             10                5 2019-10-01

[18064 rows x 2 columns]

In [12]:
df_set_synt = df_set.loc[df_set['TyAr'] == 'ZPF'][['Article', 'Composant']].set_index('Composant')

In [13]:
len(set(df_set_synt.index))

272

In [237]:
def get_correl_df(df_grouped_cite, country, print_ok = False):
    lags = [-2,-1,0,1,2]
    columns_names = ['Composant','Set Parent'] + ['Correlation Lag ' + str(lag) for lag in lags] + ['Correl Max', 'Lag with Correl Max']
    df_all_correl = pd.DataFrame(columns = columns_names)
    
    cites_country = set(df_grouped_cite.loc[country].index.get_level_values(0))
    
    for cite in set(df_set_synt.index):

        df_for_correl = pd.DataFrame()


        if cite in cites_country:
             
            fig = go.Figure()
            fig.update_layout(height=400, width=1000)
            fig.update_layout(
            yaxis_title='Sales_Quantity',
            xaxis_title="Date")

            df_cite = df_grouped_cite.loc[(country,cite)].set_index('Date')
            date_index = pd.date_range(start=df_cite.index[0], periods=1, freq='M')
            df_for_correl = pd.DataFrame(index = date_index)
            fig.add_trace(go.Scatter(x=df_cite.index, y=df_cite['Q_BIL_GROSAL'], name = 'Composant ' + cite,mode='lines + markers' ))

            df_for_correl = pd.concat([df_for_correl,df_cite])


            cite_set =   list(df_set_synt.loc[cite]['Article'])


            flag = False
            for cite_set_i in cite_set:
                    if cite_set_i in cites_country:

                        df_cite_set = df_grouped_cite.loc[(country,cite_set_i)].set_index('Date')

                        df_cite_set=df_cite_set.rename(columns = {'Q_BIL_GROSAL': cite_set_i})


                        correls = []

                        df_for_correl = pd.concat([df_for_correl,df_cite_set],axis = 1)
                        lag_with_correl_max = np.nan
                        correl_max = 0
                        for lag in lags:

                            df_for_correl[cite_set_i + '_shifted_'+ str(lag)] = df_for_correl[cite_set_i].shift(lag)




                            if df_for_correl[[cite_set_i + '_shifted_'+ str(lag),'Q_BIL_GROSAL']].dropna().shape[0] > 6:

                                start = df_for_correl[cite_set_i + '_shifted_'+ str(lag)].dropna().index[0]
                                end = df_for_correl[cite_set_i + '_shifted_'+ str(lag)].dropna().index[-1]


                                df_for_correl = df_for_correl.loc[start:end]
                                df_for_correl = df_for_correl.fillna(0)

                                correl = df_for_correl['Q_BIL_GROSAL'].corr(df_for_correl[cite_set_i + '_shifted_'+ str(lag)],'pearson')
                                correls.append(correl)

                                if abs(correl) > abs(correl_max):
                                    #print(correl, correl_max)
                                    lag_with_correl_max = lag
                                    correl_max = correl
                            else:
                                correls.append(np.nan)

                        if abs(correl_max) > 0.5:
                            flag = True
                            fig.add_trace(go.Scatter(x=df_for_correl.index, y=df_for_correl[cite_set_i + '_shifted_'+ str(lag_with_correl_max)], 
                                                     mode='lines + markers', 
                                                     name = 'Set ' + cite_set_i + '_shifted_'+ str(lag_with_correl_max) + '  with R2 : ' 
                                                     + str(round(correl_max,2))))



                        df_all_correl = df_all_correl.append(pd.DataFrame([[cite, cite_set_i] + correls +[correl_max,lag_with_correl_max]],
                                                                      columns = columns_names))

            if flag and print_ok == True:
                fig.write_image(cite + ".png")
                fig.show()
    
    df_all_correl = df_all_correl.set_index(['Composant','Set Parent'])
    return df_all_correl

In [235]:
country = 'TR Asia'
df_all_correl = get_correl_df(df_grouped_cite,country,False)

In [228]:
#df_all_correl_sorted = df_all_correl.sort_values('Max Correl').drop(columns = ['Max Correl','Min Correl'])

#fig = go.Figure(data=go.Heatmap( 
#    x=[-2,-1,0,1,2],
#                   z=df_all_correl_sorted))
#fig.show()

In [243]:
def get_stats(df_all_correl, lim):
    total_unique_couples = df_all_correl.shape[0]
    with_enough_values = total_unique_couples - df_all_correl['Correlation Lag 0'].isna().sum()
    df_interest = df_all_correl.loc[abs(df_all_correl['Correl Max']) > lim]
    with_1correl_sup06 = df_interest.shape[0]
    nb_correl_synchrone = df_interest.loc[(df_interest['Lag with Correl Max'] == 0) & (df_interest['Correl Max'] > 0)].shape[0]
    nb_cannibalisation_synchrone = df_interest.loc[(df_interest['Lag with Correl Max'] == 0) & (df_interest['Correl Max'] < 0)].shape[0]
    nb_follower_suiveur  = df_interest.loc[(df_interest['Lag with Correl Max'].isin([1,2,-1,-2])) & (df_interest['Correl Max'] > 0)].shape[0]
    nb_cannibalisation_asynchrone = df_interest.loc[(df_interest['Lag with Correl Max'].isin([1,2,-1,-2])) & (df_interest['Correl Max'] < 0)].shape[0]
    df_summary = pd.DataFrame([total_unique_couples,with_enough_values,with_1correl_sup06,nb_correl_synchrone,
                       nb_cannibalisation_synchrone,nb_follower_suiveur,nb_cannibalisation_asynchrone],
                       index=['Nb Unique Couple', 'With Enough Value', 'Presenting a Correlation > ' + str(lim)+ ' ', 
                              'Correl Synchrone','Cannibalisation','Follower-Suiveur','Cannibalisation décalée'], columns=[country])
    return df_summary

In [232]:
df_summary = get_stats(df_all_correl, lim =0.55)

In [233]:
df_summary

,TR Asia
Nb Unique Couple in TR Asia,76
With Enough Value,53
Presenting a Correlation > 0.55,30
Correl Synchrone,12
Cannibalisation,2
Follower-Suiveur,10
Cannibalisation décalée,6


In [252]:
df_summary_all = pd.DataFrame()
for country in ['TR Asia', 'TR Europe','France Boutiques']:
    print(country)
    df_all_correl = get_correl_df(df_grouped_cite,country,False)
    df_summary = get_stats(df_all_correl, lim =0.55)
    if df_summary.loc['With Enough Value'][country] > 0:
        df_summary_all = pd.concat([df_summary_all,df_summary], axis = 1)
df_summary_all

TR Asia
TR Europe
France Boutiques


,TR Asia,TR Europe,France Boutiques
Nb Unique Couple,76,60,57
With Enough Value,53,38,30
Presenting a Correlation > 0.55,30,14,14
Correl Synchrone,12,1,0
Cannibalisation,2,0,1
Follower-Suiveur,10,7,11
Cannibalisation décalée,6,6,2


In [253]:
df_summary_all.to_excel('Correl_Analyses.xlsx')